## Счёты

--------------------------------------------------------------------------------
Данная тетрадь является попыткой смоделировать вычисление корня
*Вавилонским методом* на счётах применяя операции *НОД*, *НОК* и *факторизацию*
исключительно счётным целочисленным методом.

In [294]:
import numpy as np
import pandas as pd

In [295]:
class Row():
    def __init__(self, period, name='a'):
        # Размер порядка системы счисления
        # Для десятичной системы нужно передать число 10
        # При этом максимальное количество элементов будет 9
        self.period = period
        self.data = []
        self.dataCopy = []
        self.carry = False
        
        self.name = name
        self.Q = 0

    def preserve(self):
        self.dataCopy = self.data.copy()

    def restore(self):
        self.data = self.dataCopy
    
    def hasItems(self):
        assert(len(self.data) < self.period)
        assert(len(self.data) >= 0)

        #print(self.name + ' hasItems: ' + str(len(self.data)))
        
        return len(self.data) > 0

    def isEmpty(self):
        assert(len(self.data) < self.period)
        assert(len(self.data) >= 0)
        return len(self.data) == 0

    def isFull(self):
        assert(len(self.data) < self.period)
        return len(self.data) == self.period - 1

    def clear(self):
        self.data.clear()
        self.dataCopy.clear()

    def isCarry(self):
        return self.carry

    def clearCarry(self):
        self.carry = False

    def fill(self):
        assert(len(self.data) < self.period)
        while(not self.isFull()):
            self.data.append(1)

    def push(self):
        assert(len(self.data) < self.period - 1)
        self.data.append(1)
    
    def pop(self):
        assert(len(self.data) < self.period)
        assert(len(self.data) > 0)
        # На счётах могут быть только камушки
        assert(self.data.pop() == 1)
    
    def add(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)
        assert(len(self.data) < self.period)
        self.carry = False

        row.preserve()
        while(row.hasItems()):
            if (self.isFull()):
                self.carry = True
                self.data.clear()
            row.pop()
            self.data.append(1)
        row.restore()

    def sub(self, row):
        #print('---- ' + self.name)
        #print('self: ' + str(self.getDecimal()))
        #print('row : ' + str(row.getDecimal()))
        #print('---- ' + row.name)

        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)
        assert(len(self.data) < self.period)
        self.carry = False

        row.preserve()
        #print('before')
        while(row.hasItems()):
            #print(row.getDecimal())
            #print(str(row.hasItems()))
            #print(self.name + ' while(row.hasItems())')
            if (self.Q > 100):
                assert(False)
            else:
                self.Q = self.Q + 1

            #print('----')
            #print(row.getDecimal())
            row.pop()
            #print(row.getDecimal())
            
            if (self.hasItems()):
               # print('one')
                self.pop()
            else:
                #print('two')
                self.carry = True
                self.fill()
        #print('row.restore()')
        row.restore()

    # Больше чем переданная строка
    def isBigger(self, row):
        assert(type(row) == Row)
        assert(self is not row)
        assert(self.period == row.period)

        row.preserve()
        #print('row.sub(self)')
        row.sub(self)
        row.restore()

        return row.isCarry()
    
    def toArray(self):
        assert(len(self.data) < self.period)
        arr = np.zeros(self.period - 1, dtype=int)
        self.preserve()
        i = 0
        while(self.hasItems()):
            arr[i] = self.data.pop()
            i = i + 1
        self.restore()
        return arr
    
    def setDecimal(self, num):
        assert(type(num) == int)
        # Не будем здесь поднимать флаг заёма
        assert(num < self.period)
        for i in range(num):
            self.data.append(1)

    def getDecimal(self):
        return len(self.data)

In [298]:
r1 = Row(10)
r2 = Row(10)
r3 = Row(10)

r1.setDecimal(2)
r2.setDecimal(4)
r3.setDecimal(7)

assert(r1.getDecimal() == 2)
assert(r2.getDecimal() == 4)
assert(r3.getDecimal() == 7)

r2.add(r1)
assert(r2.getDecimal() == 6)
assert(r1.getDecimal() == 2)

r3.sub(r2)
assert(r3.getDecimal() == 1)
assert(r2.getDecimal() == 6)

assert(r2.isBigger(r3))
assert(not r1.isBigger(r2))

r2.setDecimal(2)
assert(not r1.isBigger(r2))

In [299]:
class Abak():
    def __init__(self, shape, name='a'):
        assert(type(shape) == tuple)
        assert(len(shape) == 2)
        periods = shape[0]
        period = shape[1]
        assert(periods > 0)
        assert(period > 0)
        self.shape = shape
        self.maxValue = (period ** periods) - 1
        self.rows = [Row(period, name) for i in range(periods)]

    def add(self, abak):
        assert(type(abak) == Abak)
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])
        
        periods = abak.shape[0]
        isCarry = False
        for i in range(periods):
            if (isCarry):
                self.rows[i].push()
            self.rows[i].add(abak.rows[i])
            isCarry = self.rows[i].isCarry()

    # Больше чем переданная таблица
    def isBigger(self, abak):
        assert(type(abak) == Abak)
        assert(self is not abak)
        assert(self.shape[1] == abak.shape[1])

        periodSelf = self.shape[0] - 1
        periodOther = abak.shape[0] - 1

        #print('Indexes start')
        #print(periodSelf, periodOther)
        
        while(self.rows[periodSelf].isEmpty() and periodSelf >= 0):
            periodSelf = periodSelf - 1
        while(abak.rows[periodOther].isEmpty() and periodOther >= 0):
            periodOther = periodOther - 1

        #print('Indexes finish')
        #print(periodSelf, periodOther)
        
        if (periodSelf < 0 and periodOther < 0):
            return False
        elif (periodSelf > periodOther):
            return True
        elif (periodSelf < periodOther):
            return False
        else:
            rowSelf = self.rows[periodSelf]
            rowOther = abak.rows[periodOther]
            return rowSelf.isBigger(rowOther)
            
    
    def sub(self, abak):
        assert(type(abak) == Abak)
        assert(self.shape[0] >= abak.shape[0])
        assert(self.shape[1] == abak.shape[1])
        
        for i in range(periods):
            amount = self.rows[i].getDecimal()
            num = num + (period ** i) * amount

        return num
                
    def toFrame(self):
        rows = self.shape[0]
        cols = self.shape[1] - 1
        table = np.zeros((rows, cols), dtype=int)
        for i in range(rows):
            table[i] = self.rows[i].toArray()
        return pd.DataFrame(
            table,
            columns=[str(i + 1) for i in range(cols)]) 

    def setDecimal(self, num):
        assert(type(num) == int)
        assert(num > 0)
        assert(num <= self.maxValue)

        periods = self.shape[0]
        for i in range(periods):
            self.rows[i].clear()
            
        if (num == 0):
            return

        i = 0
        period = self.shape[1]
        # Ищем граничный период
        while((period ** i) <= num):
            i = i + 1

        for j in reversed(range(i)):
            amount = period ** j
            while((num - amount) >= 0):
                self.rows[j].push()
                num = num - amount

    def getDecimal(self):
        periods = self.shape[0]
        period = self.shape[1]

        num = 0
        for i in range(periods):
            amount = self.rows[i].getDecimal()
            num = num + (period ** i) * amount

        return num

In [301]:
# Передаём количество периодов и размер периода
a1 = Abak((4, 10))
a2 = Abak((4, 10), 'b')
a1.setDecimal(100)
a2.setDecimal(4321)
assert(a1.getDecimal() == 100)
assert(a2.getDecimal() == 4321)
assert(a2.isBigger(a1))
assert(not a1.isBigger(a2))

a2.add(a1)
assert(a2.getDecimal() == 4421)
assert(a1.getDecimal() == 100)

a2.setDecimal(100)
assert(not a1.isBigger(a2))

a1.setDecimal(4321)
assert(a1.getDecimal() == 4321)
a1.toFrame()

,1,2,3,4,5,6,7,8,9
0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0


In [ ]:
np.zeros(5)

In [ ]:
[i for i in range(1)]

In [ ]:
len((1,2))